# Using the calf package

The calf package includes a `main()` function, which is the main way to execute it.

We start by doing some preparation:

- Create a datacube client
- Load the region of interest and crop mask

In [ ]:
%matplotlib inline
import logging
import os
from pathlib import Path


import datacube
import geopandas
import ipywidgets as widgets
import numpy as np
import rasterio.windows
import rioxarray
import xarray as xr
from IPython.display import display

import calf

# this is a minor detail, toning down third-party loggers
logging.basicConfig(level=logging.INFO)
for logger_name in ("datacube", "fiona", "matplotlib", "pyproj", "rasterio"):
    logging.getLogger(logger_name).setLevel(logging.WARNING)


dc = datacube.Datacube(
    app="test-calf",
    env="sandbox",
    config=os.getenv("DATACUBE_CONFIG_PATH")
)

roi_gdf = geopandas.read_file(Path("../test-data/auxiliary.gpkg"), layer="region-of-interest")
crop_gdf = geopandas.read_file(Path("../test-data/auxiliary.gpkg", layer="crop-mask"))

roi_gdf.plot()
crop_gdf.plot()

In [ ]:
output_resolution = 20
start_date = "2015-01-01"
end_date = "2021-12-31"

calf_result = calf.compute_calf(
    dc, 
    start_date=start_date,
    end_date=end_date,
    ard_product="test_spot7_gauteng_old_eo3",
    region_of_interest_gdf=roi_gdf,
    crop_mask_gdf=crop_gdf,
    vegetation_threshold=0.2,
    red_band="red",
    nir_band="nir",
    qflags_band="spclass",
    output_crs="EPSG:32635",
    output_resolution=output_resolution,
    return_patches=True
)

The output of the function is an object of type `calf.CalfAlgorithmResult`. It contains the following properties:

- `calf_ds` - an `xarray.Dataset` that contains the final spatial layers generated by the CALF algorithm
- `calf_stats` - a `pandas.DataFrame` that contains the statistics gathered during calculation
- `patches` - a list with the individual raster patches that were processed. **NOTE** - In order to have this list be populated with the actual patches, you must provide `return_patches=True` as a parameter to the calf algorithm. Otherwise it will be an empty list. This should be used for debug purposes only, as it increases the algorithm's memory consumption

## calf_ds

the output `calf_ds` is an `xarray.Dataset` that contains the CALF spatial layers. The following are included:

- `calf` - This is the final CALF product, reclassified
- `raw_calf` - This is the non-reclassified CALF product
- `seasonal_ndvi` - This is the aggregated NDVI that served as an input for computing CALF

In [ ]:
calf_result.calf_ds

### calf

This data array contains the final CALF values

In [ ]:
calf_result.calf_ds.calf.plot(levels=[0, 1, 2, 100, 101])

### raw_calf

This data array contains the un reclassified CALF values

In [ ]:
calf_result.calf_ds.raw_calf.plot()

### seasonal_ndvi

This data array contains the aggregated NDVI

In [ ]:
calf_result.calf_ds.seasonal_ndvi.plot()

## calf_stats

This is a `pandas.DataFrame` that contains the statistic values gathered when running the algorithm

In [ ]:
calf_result.calf_stats

## Saving results

In order to save results to disk you can use the following auxiliary functions:

In [ ]:
stats_output_path = Path(f"../results/calf_stats_{start_date}-{end_date}_{output_resolution}m.csv")
calf_tiff_output_path = Path(f"../results/calf_{start_date}-{end_date}_{output_resolution}m.tif")
calf_aux_tiff_output_path = Path(f"../results/calf-aux_{start_date}-{end_date}_{output_resolution}m.tif")


calf.save_calf_result(calf_result.calf_ds, calf_tiff_output_path)
calf.save_aux_calf_result(calf_result.calf_ds, calf_aux_tiff_output_path)

calf_result.calf_stats.to_csv(stats_output_path, index=False)